In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl

In [ ]:
import gensim.downloader as dl
from gensim.models import KeyedVectors

In [ ]:
pretrained_weights_name = "word2vec-google-news-300"
model_dl_path = os.path.join(
    dl.BASE_DIR, pretrained_weights_name, f"{pretrained_weights_name}.gz")


if os.path.exists(model_dl_path):
    # load model
    print(f"Loading model from {model_dl_path}")
    gnews_embeddings = dl.load(pretrained_weights_name)
else:
    # download
    print(f"Model will be downloaded at {model_dl_path}")
    gnews_embeddings = dl.load("word2vec-google-news-300")


In [ ]:
vocabulary = gnews_embeddings.index_to_key  # type: ignore
vocab_len = len(vocabulary)
vocab_len

In [ ]:
# https://github.com/Oneplus/Tweebank

train_file = os.path.join(
    "/mnt/Others/experiments/datasets/Tweebank-dev/converted/"
    "en-ud-tweet-train.fixed.conllu")

# assert os.path.exists(train_file)

with open(train_file) as f:
    data = f.readlines()

# data

In [ ]:
# break line at every "\n"
tweets = list()
buffer = list()
for idx, tw in enumerate(data):
    if tw == "\n":
        # one partition here
        tweets.append(buffer)
        buffer = []
    else:
        # keep appending
        buffer.append(tw)
        
tweets[0]

In [ ]:
# format for tokens
# number - word - lemma - pos - _ - _ - id - role, -, - 

'4\tDear\tdear\tADJ\t_\t_\t5\tamod\t_\t_\n'.split("\t")


In [ ]:
# need idx 1, 2,3 : word, lemma and pos

class ConlluRowInfo:
    word: str
    lemma: str
    pos: str
    
    def __init__(self, word: str, lemma: str, pos: str) -> None:
        self.word = word
        self.lemma = lemma
        self.pos = pos
        
    def __str__(self) -> str:
        rep = {
            "word": self.word,
            "lemma": self.lemma,
            "pos": self.pos
        }
        return str(rep)

In [ ]:
from typing import List

class ConlluRow:
    info: List[ConlluRowInfo]
    # text: str
    
    def __init__(self, infos: List[ConlluRowInfo]) -> None:
        self.info = infos
        
    def __str__(self) -> str:
        return f"info : {self.info}"

In [ ]:
structured_tweets = list()

for tweet in tweets:
    # text = tweet[1].replace("# text = ", "")
    info_in_tweet = list()
    for infos in tweet[2:]:
        buffer = infos.split("\t")
        try:
            word = buffer[1]
            lemma = buffer[2]
            tag = buffer[3]
            info_in_tweet.append(ConlluRowInfo(word, lemma, tag))
        except IndexError:
            print(buffer)
        except AttributeError as e:
            print(e.name)
    structured_tweets.append(ConlluRow(info_in_tweet))  

In [ ]:
# time to define the torch dataset

from torch.utils.data import Dataset
from tqdm.auto import trange, tqdm
from typing import Dict, List

class TweebankDataset(Dataset):
    def __init__(self, filename, w2v_weights=gnews_embeddings) -> None:
        self.filename = filename
        
        self.w2v = w2v_weights
        self.data = list()
        self.__read_data()
        
        self.MAX_SEQ_LEN = 50 # default value
        # self.__find_max_seq_len()
        
        self.UNIQUE_TAGS = ['PRON', 'NUM', 'NOUN', 'CCONJ', 'ADV', 'SCONJ', 
                               'ADP', 'AUX', 'PROPN', 'SYM', 'DET', 
                               'INTJ', 'PUNCT', 'X', 'ADJ', 'VERB', 'PART']
        self.tag_dict = dict()
        self.__encode_tags()
        
        self.number_tags = len(self.UNIQUE_TAGS)
        
        self.vocabulary = self.w2v.index_to_key  # type: ignore
            
    
    def __len__(self) ->  int:
        return len(self.data)
    
    def __getitem__(self, idx) -> Dict[str, torch.Tensor]:
        # ============== collect ===================
        words = [i.word for i in self.data[idx].info]
        # lemmas = [i.lemma for i in self.data[idx].info]
        tags = [i.pos for i in self.data[idx].info]
        
        
        # =================== convert using word2vec weights ==========
        for idx in range(len(words)):
            try:
                w2v_idx = self.w2v.key_to_index[words[idx]]  # type: ignore 
            except KeyError:
                w2v_idx = 0 # </s>
            words[idx] = w2v_idx
            tags[idx] = self.tag_dict[tags[idx]]
            
        
        # ============== pad words ===============
        # left pad
        padded_words = np.zeros(self.MAX_SEQ_LEN, dtype=np.int32)
        padded_words[-len(words):] = words
        
        padded_tags = np.zeros(self.MAX_SEQ_LEN, dtype=np.int32)
        padded_tags[-len(tags):] = tags
        
        return {
            "words": torch.tensor(padded_words),
            # "lemmas": lemmas,
            "tags": torch.tensor(padded_tags),
            "non_padded_len": torch.tensor(len(tags))
        }
        
    def __find_max_seq_len(self) -> None:
        seq_lens = []
        
        for idx in range(len(self.data)):
            words = [i.word for i in self.data[idx].info]
            seq_lens.append(len(words))
        
        
        self.MAX_SEQ_LEN = max(seq_lens)
        
    def __encode_tags(self) -> None:
        for idx, tag in enumerate(self.UNIQUE_TAGS):
            self.tag_dict[tag] = idx
        
    def __read_data(self) -> None:
        with open(self.filename, "r") as f:
            data = f.readlines()
            
            # ============ read the text file =============
            lines = list()
            buffer = list()
            for _, line in tqdm(enumerate(data)):
                if line == "\n":
                    lines.append(buffer)
                    buffer = []
                else:
                    buffer.append(line)
                    
            # ============== organize in objects ==============
            for idx, line in tqdm(enumerate(lines)):
                # from line index 2 and onwards
                line_info = list()
                for info in line[2:]:
                    buffer = info.split("\t")
                
                    try:
                        word = buffer[1]
                        lemma = buffer[2]
                        tag = buffer[3]
                        
                        line_info.append(ConlluRowInfo(word, lemma, tag))
                        
                    except IndexError:
                        print(buffer)
                        
                
                lines[idx] = ConlluRow(line_info)    

            self.data = lines


In [ ]:
dataset = TweebankDataset(train_file)
sample = dataset[0]

In [ ]:
sample

In [ ]:
# https://stackabuse.com/python-how-to-flatten-list-of-lists/


# import itertools

# all_tags = [data["tags"] for data in dataset]
# all_tags = list(itertools.chain(*all_tags))
# unique_tags = set(all_tags)
# print(list(unique_tags))

In [ ]:
# model
# https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim: int, hidden_dim: int,  tagset_size: int, freeze_embeddings=True, w2v_weights=gnews_embeddings) -> None:
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.taget_size = tagset_size
        self.freeze_embeddings = freeze_embeddings
        # self.save_hyperparameters() 
        
        
        embedding_tensors = torch.from_numpy(w2v_weights.vectors) # type: ignore        
        self.word_embeddings = nn.Embedding.from_pretrained(
            embedding_tensors, freeze=self.freeze_embeddings, padding_idx=0)
        self.lstm = nn.LSTM(
            embedding_dim, 
            hidden_dim, 
            batch_first=True,
            bidirectional=True)
        self.attention =  nn.MultiheadAttention(hidden_dim * 2, num_heads=4, dropout=0.1, batch_first=True)
        self.linear = nn.Linear(hidden_dim * 2, tagset_size)
        self.relu = nn.ReLU()
        
    def forward(self, words):
        embeds = self.word_embeddings(words)
        lstm_out, _ = self.lstm(embeds)
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        linear_out = self.linear(attn_out)
        linear_out = self.relu(linear_out)
        logits = F.log_softmax(linear_out, dim=-1)
        return logits
        

In [ ]:
model = LSTMTagger(embedding_dim=300, hidden_dim=100,  tagset_size=dataset.MAX_SEQ_LEN)

# run a sample forward pass
sample = dataset[42]
out = model(sample["words"])
out.size()

In [ ]:
out

In [ ]:
torch.max(out, dim=0)

In [ ]:
sample["tags"]

In [ ]:
device = torch.device("cuda")
model = model.to(device)

In [ ]:
# dataloaders
from torch.utils.data import DataLoader

bs = 128
dl_args = {
    "pin_memory": True,
    "batch_size": bs
}


training_set = dataset
validation_set = TweebankDataset("/mnt/Others/experiments/datasets/Tweebank-dev/converted/en-ud-tweet-dev.fixed.conllu")
test_set = TweebankDataset("/mnt/Others/experiments/datasets/Tweebank-dev/converted/en-ud-tweet-test.fixed.conllu")

train_loader = DataLoader(training_set, shuffle=True, **dl_args)
val_loader = DataLoader(validation_set, shuffle=False, **dl_args)
test_loader = DataLoader(test_set, shuffle=False, **dl_args)

In [ ]:
optimizer = optim.AdamW(params=model.parameters())
criterion = nn.NLLLoss(ignore_index=0)
run_validation_every_n_step = 100


# fp16
scaler = torch.cuda.amp.GradScaler()

epochs = 300
for e in trange(epochs):

    steps = 0
    for batch in train_loader:
        # switch to train mode
        model.train()
        
        words = batch["words"]
        tags = batch["tags"].long()
        
        # send data to device
        words = words.to(device)
        tags = tags.to(device)
        
        # zero out optimizer to accumulate new grads
        optimizer.zero_grad()
        
        with torch.autocast(device_type="cuda", dtype=torch.float16):
            logits = model(words)
            # loss
            loss = criterion(logits, tags)
        
        
        # ======== validation ==============
        if steps % run_validation_every_n_step == 0:
            val_losses = []
            
            # switch context
            model.eval()
            with torch.no_grad():
                for val_batch in val_loader:
                    words = val_batch["words"]
                    tags = val_batch["tags"].long()
                    
                    words = words.to(device)
                    tags = tags.to(device)
                    
                    with torch.autocast(device_type="cuda", dtype=torch.float16):
                        logits = model(words)
                        val_loss = criterion(logits, tags)

                    val_losses.append(val_loss.item())
                    
                    # preds = torch.max(logits, dim=-1).indices

                # log
                print(f"Epoch:: [{e + 1}]/[{epochs}] Step:: {steps}")
                print(f"Train Loss:: {loss} __________ Val Loss:: {torch.mean(torch.tensor(val_losses))}")
        
        # switch context
        model.train()
        scaler.scale(loss).backward()  # type: ignore
        # loss.backward()
        scaler.step(optimizer)
        # optimizer.step()
        scaler.update()
        steps += 1
    


In [ ]:
def accuracy(pred:torch.Tensor, true: torch.Tensor, non_pad_start: torch.Tensor) -> torch.Tensor:

    
    non_padded_pred = (pred != torch.tensor(0)).nonzero()
    non_padded_true = (true != torch.tensor(0)).nonzero()
    
    correct = pred[non_padded_pred].eq(true[non_padded_true])
    return correct.sum() / true.size()[1]

In [ ]:
acc = []
model.eval()
with torch.no_grad():
    for batch in tqdm(train_loader):
        words = batch["words"].to(device)
        tags = batch["tags"].long().to(device)
        non_pad_start = batch["non_padded_len"].to(device)
        
        
        logits = model(words)
        
        preds = torch.max(logits, dim=0).indices
        
        a = accuracy(preds, tags, non_pad_start)
#         print(a)
        
        acc.append(a)

print(f"Mean acc : {torch.mean(torch.tensor(acc))}")
        